In [1]:
#安裝套件
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import nltk
import re
from nltk.tokenize import TweetTokenizer
import numpy as np
from stemming.porter2 import stem


knzr = TweetTokenizer()
tokenizer = RegexpTokenizer(r'\w+')
en_stop = get_stop_words('en') 
lemma = nltk.wordnet.WordNetLemmatizer()

In [2]:
from spacy.en import English
nlp = English()

In [3]:
#電影data
from collections import Counter

import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_IMDB;UID=sa;PWD=scucc')
cursor = conn.cursor()
cursor.execute("select * from Moviedata where   [movieNameE]='Ant-Man' ")
rows = cursor.fetchall()



Stars= rows[0][-1].split(',')
Writer=rows[0][-2].split(',')
Director=rows[0][-3].split(',')

MovieName=rows[0][1]
StarsDictionary=[]
WritersDictionary=[]
DirectorsDictionary=[]
for Star in Stars:
    Ssplit= Star.strip().split(' ')
    StarsDictionary.append(Ssplit)
for Star in Writer:
    Ssplit= Star.strip().split(' ')
    WritersDictionary.append(Ssplit)
for Star in Director:
    Ssplit= Star.strip().split(' ')
    DirectorsDictionary.append(Ssplit)
    
print DirectorsDictionary

[[u'Peyton', u'Reed']]


In [4]:
#FBPost搜尋
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_IMDB;UID=sa;PWD=scucc')
cursor = conn.cursor()
cursor.execute("select * from FBPost  where  [message] like '%Ant-Man%trailer%'  ")
rows = cursor.fetchall()
PostNO=rows[-2][0]
Like=rows[-2][3]
Coment=rows[-2][4]
print PostNO

2517


In [5]:
#Aspect搜尋
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_IMDB;UID=sa;PWD=scucc')
cursor = conn.cursor()
cursor.execute("select * from [Aspect] ")
AspectRows = cursor.fetchall()
print AspectRows[0]

(u'OA', u'accept', u'accept')


In [32]:
#Opinion搜尋
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_IMDB;UID=sa;PWD=scucc')
cursor = conn.cursor()
cursor.execute("select * from [Opinion] ")
OpinionRows = cursor.fetchall()
print OpinionRows[0]

(u'simple', 1.0)


In [7]:
#Emotion SQLSEVER
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_IMDB;UID=sa;PWD=scucc')
cursor = conn.cursor()
cursor.execute("select * from [Emotion] ")
EmotionRows = cursor.fetchall()


def Emotion_Detect(Word):
    P=0
    N=0
    for EmotionRow in EmotionRows:
        
        if EmotionRow[2] in Word:
            if EmotionRow[4]==1 :
                P+=1
                #print Word
                break
            if EmotionRow[4]==-1:
                N+=1
                #print Word
    EmotionArray=[P,N]
    return  np.array(EmotionArray)
    
 



In [36]:
#Slang SQLSEVER
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_IMDB;UID=sa;PWD=scucc')
cursor = conn.cursor()
cursor.execute("select * from Slang ")
SlangRows = cursor.fetchall()

def Slang_Detect(Word):
    
    P=0
    N=0
    for SlangRow in SlangRows:
        if SlangRow[0].lower() == Word:
            
            if SlangRow[3]==1 :
                P+=1
                SlangPWord[Word]+=1
                
                break
            if SlangRow[3]==-1:
                N+=1
                SlangNWord[Word]+=1
                break
            
    SlangArray=[P,N]
    return  np.array(SlangArray)
print Slang_Detect('lol')


[1 0]


In [9]:
def preprocessEmotion(Comment):
#     print "原始"
#     print Comment
    Comment= Comment.lower()
#     print "轉小寫"
#     print Comment
    sent=re.split(r'[,.!?]+', Comment)
#     print "斷句"
#     print sent
    for i in range (0,len(sent)):
        toeken1= sent[i].split(" ")
        sent[i]=toeken1
#     print "依空白切字"
#     print sent   
    stop(sent)
#     print "去除STOPWORD"
#     print sent
    return sent

In [10]:
def preprocessSlang(Comment):

    Comment= Comment.lower()
    sent=re.split(r'[,.!?]+', Comment)
    for i in range (0,len(sent)):
        toeken1= sent[i].split(" ")
        sent[i]=toeken1
  
    stop(sent)
    repunctForSlang(sent)

    for n in sent:
        while "" in n:
            n.remove("")

    return sent

In [11]:
def preprocessALL(Comment):
    Comment= Comment.lower()

    sent=re.split(r'[,.!?]+', Comment)

    for i in range (0,len(sent)):
        toeken1= sent[i].split(" ")
        sent[i]=toeken1
    stop(sent)
    repunct(sent)
#     print "去掉符號"
#     print sent
    
    
     #去掉沒東西的元素
    for n in sent:
        while "" in n:
            n.remove("")
#     print "去掉空白"
#     print sent
    return sent

In [25]:
def adjOnly(Comment):
    
    sentence=unicode(Comment)
    tokens = nlp(sentence)
    newComment=""
    for tok in tokens:
        
        if tok.pos_=='ADJ'or tok.pos_=='VERB'or tok.pos_=='INTJ':
            newComment+=str(tok)+" "
    return newComment 
print adjOnly('I love good you')

love good 


In [26]:
def NounOnly(Comment):
    
    sentence=unicode(Comment)
    tokens = nlp(sentence)
    newComment=""
    for tok in tokens:
        
        if tok.pos_=='PROPN'or tok.pos_=='PRON'or tok.pos_=='NOUN'or tok.pos_=='verb':
            newComment+=str(tok)+" "
    return newComment 
print NounOnly('I love good Annie movie')

I Annie movie 


In [38]:
#CommentOpinion搜尋
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_IMDB;UID=sa;PWD=scucc')
cursor = conn.cursor()
cursor.execute("select * from CommentOpinion  where  [PostNO]=? ",PostNO)
rows = cursor.fetchall()

SlangPWord=Counter()
OAWord=Counter()
SEWord=Counter()
STWord=Counter()
StarWord=Counter()
WriterWord=Counter()
SlangNWord=Counter()
OpinionPWord=Counter()
OpinionNWord=Counter()


Opinion=Counter()
Emotion=Counter()

starCount=0
ALLCountArray=[0,0,0,0,0]
EmotionArray=[0,0]
SlangArray=[0,0]
ALLCountArray=np.array(ALLCountArray)
EmotionArray=np.array(EmotionArray)
SlangArray=np.array(SlangArray)
for row in rows:
    P_N=0
    
    Comment= row[3].lower()
    
    
    
    
    
    #Comment='it seems like every Marvel movie has to have f***ing evil robots to complete the plot. thanks but no thanks, waiting for next superman movie.'
    
    sent=preprocessEmotion(Comment)
    
    for sentNO in range (0,len(sent)):
        for word in range (0,len(sent[sentNO])):
            Emotion= Emotion_Detect(sent[sentNO][word])
            EmotionArray+=Emotion
            
    sent= preprocessSlang(Comment)      
            
    for sentNO in range (0,len(sent)):
        for word in range (0,len(sent[sentNO])):
            Word=stem(sent[sentNO][word])
            Slang= Slang_Detect(Word)
            SlangArray+=Slang
            
            
            
    
    Comment= row[3]
    Comment=adjOnly(Comment)
    sent=preprocessALL(Comment)
    
    for sentNO in range (0,len(sent)):
        for word in range (0,len(sent[sentNO])):
           
            Word=stem(sent[sentNO][word])
            P_N= opinionDetect2(Word)
            if P_N==1:
                Opinion['pos']+=1
            if P_N==-1:
                Opinion['neg']+=1
                
                
    Comment= row[3].lower()
    Comment=NounOnly(Comment)
    
    sent=preprocessALL(Comment)
                
    for sentNO in range (0,len(sent)):
        for word in range (0,len(sent[sentNO])):

            Word=stem(sent[sentNO][word])       
            
            AspectArray= AspectDetect(Word)
            ALLCountArray+=AspectArray
            
            
            
            
    Comment= row[3].lower()
    
    
    NameArray=Namedetect(Comment)
    ALLCountArray+=NameArray
    
    
print ALLCountArray    
   
print "OA:"
print OAWord.most_common()
print "\n"

print "SE:"
print SEWord.most_common()
print "\n"

print "ST:"
print STWord.most_common()
print "\n"

print "Opinion:"
print Opinion
print "\n"

print "Emotion:"
print EmotionArray
print "\n"

print "Writer:"
print WriterWord.most_common()
print "\n"

print "Star:"
print StarWord.most_common()
print "\n"

print "撞出意見結果"
print "正"
print OpinionPWord.most_common()
print "負"
print OpinionNWord.most_common()
print "\n"
print "Slang(positive):"
print SlangPWord.most_common()
print "Slang(negtive):"
print SlangNWord.most_common()

[54 30  2  0  9]
OA:
[('movi', 30), ('film', 4), ('pick', 3), ('hit', 2), ('thank', 2), ('action', 2), ('choic', 1), ('comedi', 1), ('mind', 1), ('piec', 1), ('thought', 1), ('fail', 1), ('stuff', 1), ('hope', 1), ('comic', 1), ('danc', 1), ('problem', 1)]


SE:
[('cgi', 2)]


ST:
[('superhero', 6), ('plot', 2), ('joke', 2), ('idea', 2), ('end', 2), ('teen', 1), ('juli', 1), ('writer', 1), ('moment', 1), ('villain', 1), ('charact', 1), ('margaret', 1), ('stori', 1), ('chris', 1), ('brick', 1), ('burgundi', 1)]


Opinion:
Counter({'pos': 121, 'neg': 49})


Emotion:
[9 0]


Writer:
[(u'Edgar', 4), (u'Joe', 1)]


Star:
[(u'Paul', 6), (u'Rudd', 2), (u'Michael', 1)]


撞出意見結果
正
[('love', 20), ('good', 18), ('great', 12), ('like', 12), ('awesom', 12), ('impress', 10), ('entertain', 10), ('will', 9), ('comic', 6), ('excit', 6), ('better', 6), ('super', 5), ('funni', 5), ('know', 5), ('thought', 4), ('fantast', 4), ('amaz', 4), ('hilari', 4), ('hope', 4), ('deserv', 4), ('make', 3), ('complet',

C:\Python27\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [15]:
#偵測人名

def Namedetect(Message):
    Message.lower()
    starcount=0
    Writercount=0
    Directorcount=0
    for Stars in StarsDictionary:
        for  Star in Stars:
            if Star.lower() in Message.lower():
                
                StarWord[Star]+=1
                starcount+=1
                break
    for Stars in WritersDictionary:
        for  Star in Stars:
            if Star.lower() in Message.lower():
                WriterWord[Star]+=1

                Writercount+=1
                break
    for Stars in DirectorsDictionary:
        for  Star in Stars:
            if Star.lower() in Message.lower():
                Directorcount+=1
                break
    NameArray=[0,Writercount,0,Directorcount,starcount]
    return np.array(NameArray)


In [16]:
print Namedetect('Peyton')

[0 0 0 1 0]


In [31]:
#偵測意見

P=[]
N=[]
for OpinionRow in OpinionRows:
    if OpinionRow[1]==1:
        N.append(stem(OpinionRow[0].lower()))
    if OpinionRow[1]==0:
        P.append(stem(OpinionRow[0].lower()))
        
if ('loves' in P) :
    print 1

def opinionDetect2(word):
    P_N=0
    for pos in P:
        if word == pos:

            P_N=1
            OpinionPWord[word]+=1
    for neg in N:
        if word == neg:
            P_N=-1
            OpinionNWord[word]+=1
    return P_N    
    


In [92]:
#意見 正負表
f = open('D:/DOWNLOAD/opinion-lexicon-English/negative-words.txt', 'r')
N = f.read()
f.close()
f = open('D:/DOWNLOAD/opinion-lexicon-English/positive-words.txt', 'r')
P = f.read()
f.close()

PArray=P.split("\n")   
Array=N.split("\n")   
   
NegArray=Array[35:]
PosArray=PArray[35:]
for x in range(0,len(PosArray)):
    PosArray[x]= stem(PosArray[x].lower())
for x in range(0,len(NegArray)):
    NegArray[x]= stem(NegArray[x].lower())

def opinionDetect3(word):
    P_N=0
    for pos in PosArray:
        if word == pos:

            P_N=1
            OpinionPWord[word]+=1
    for neg in NegArray:
        if word == neg:
            P_N=-1
            OpinionNWord[word]+=1
    return P_N      
    

In [66]:
print 1

1


In [19]:

#面向偵測
def AspectDetect(word):
    OAcount=0
    STcount=0
    SEcount=0
    for  AspectRow in AspectRows:
        if  AspectRow[2] == word:
            if AspectRow[0]=='PAC' or AspectRow[0]=='PDR':
                continue
            if  AspectRow[0]=='OA':
                OAWord[word]+=1
                OAcount+=1
            if  AspectRow[0]=='ST':
                STWord[word]+=1
                STcount+=1
            if AspectRow[0]=='SE':
                SEWord[word]+=1
                SEcount+=1
    AspectArray=[OAcount,STcount,SEcount,0,0]
    return  np.array(AspectArray)
            
            


In [20]:
#去除STOP
def stop(sent):
    for sentNO in range (0,len(sent)):
        for word in range (0,len(sent[sentNO])):
            if sent[sentNO][word] in en_stop:
                sent[sentNO][word]=""

In [21]:
#去除符號
def repunct(sent):
    for sentNO in range (0,len(sent)):
        for word in range (0,len(sent[sentNO])):
            sent[sentNO][word]=re.sub(r'[^\w]', '', sent[sentNO][word])
            sent[sentNO][word]=re.sub('[0-9]', '', sent[sentNO][word])

In [22]:
#去除符號
def repunctForSlang(sent):
    for sentNO in range (0,len(sent)):
        for word in range (0,len(sent[sentNO])):
            sent[sentNO][word]=re.sub(r'[^\w]', '', sent[sentNO][word])
            

In [23]:
sentence='i love this good movie'
sentence=unicode(sentence, "utf-8")
tokens = nlp(sentence)
newComment=""
for tok in tokens:
    
    if tok.pos_=='ADJ'or tok.pos_=='VERB':
        newComment+=str(tok)+" "
print newComment

love good 


In [39]:
stem('best'.lower())

'best'